In [10]:
# !pip install python-dotenv

In [11]:
#imports
import os
import shutil
import base64
import time
import subprocess
import platform
import openai
import uuid
import asyncio
import tiktoken
import json
import re
import fitz 
import io
import hashlib
import subprocess
import nltk


In [12]:
#all from and imports
from docx import Document
from docx.shared import Inches
from datetime import datetime, timedelta 
from azure.storage.blob import BlobServiceClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField
from azure.core.credentials import AzureKeyCredential

from pptx import Presentation
nltk.download('punkt')

from collections import defaultdict
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader, UnstructuredPowerPointLoader,UnstructuredPDFLoader, TextLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.document_loaders import UnstructuredPDFLoader, PyPDFLoader
# from langchain.document_loaders import TextLoader
# from langchain.document_loaders import Docx2txtLoader
# from langchain.document_loaders import UnstructuredPowerPointLoader

from PIL import Image
from docx import Document
from pptx import Presentation

from dotenv import load_dotenv

from quart import (
    Blueprint,
    Quart,
    redirect,
    url_for,
    current_app,
    jsonify,
    make_response,
    request,
    send_from_directory,
    session,
    render_template,
    Response
)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ssirish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
# Load the .env file
load_dotenv("C:/Users/ssirish/Downloads/aiprod_code_repos/AzureOpenAIProduction/app/backend/.env_local")
# load_dotenv(dotenv_path="app\backend\.env_local")

# Constants
CHUNK_SIZE = 1000
# user_name='ssirish@dsi.com' 
pattern = r"pageno_(\d+)"
CHUNK_OVERLAP = 200
BATCH_SIZE = 100
AZURE_STORAGE_CONNECTION_STRING = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

# 'ssirish@dsi.com' = 'ssirish@dsi.com'

input_file = "C:/Users/ssirish/Downloads/Sharepoint files download/Enhertu-Overview of the DAISY Trial.docx"  # Path to your input file
output_folder = "C:/Users/ssirish/Downloads/Sharepoint files download/output"  # Path to your output folder
# output_folder = "C:/Users/a6006/OneDrive - Axtria/Documents/DSI_New/Sharepoint files download/output"  # Path to your output folder
# # output_folder = "C:\Users\a6006\OneDrive - Axtria\Documents\DSI_New\Sharepoint files download\output"  # Path to your output folder
# # output_folder = "output"  # Path to your output folder
# split_word_doc_with_images(input_file, output_folder)



Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18


In [14]:
# 'ssirish@dsi.com' = 'ssirish@dsi.com'

In [15]:
# done
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8") 
    
def img_summary(img_path):
    try:
        base64_image = encode_image(img_path)
        # print("here******47")
        openai.api_type = "azure"
        # openai.api_key = "5d69dd6f0aa14121ae81066435443abf"
        openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
        # openai.api_base = "https://openai-poc-east.openai.azure.com/"
        AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
        openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
        openai.api_version = "2024-02-15-preview"
        # print("here******52")
        response = openai.ChatCompletion.create(
                    deployment_id="GPT4o",
                    messages=[
                        {"role": "system", "content": "Act as an Image Analyzer"},
                        {"role": "user", "content": [
                            {"type": "text", "text": "Generate information that is described in the image."},
                            {"type": "image_url", "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"}
                            }
                        ]}
                    ],
                    temperature=0.0,
                    )
        # print(response['choices'][0]['message']['content'])
        # print("here******66")
        if (len(response['choices'][0]['message']['content'])>0) and (response["usage"]["total_tokens"] > 0):
            return(response['choices'][0]['message']['content'], response["usage"]["total_tokens"])
        else:
            return "", 0
    except:
        return "", 0
        pass
    #     return("""You might have asked something out of the context with the uploaded data, If not please be more specific in the input asked""", 0)
    

In [16]:
# done
def convert_docx_to_pdf(docx_file_path, output_dir=None):
    try:
    # Ensure the input file exists
        if not os.path.isfile(docx_file_path):
            raise FileNotFoundError(f"The file {docx_file_path} does not exist")
 
        # If no output directory is provided, use the same directory as the input file
        if output_dir is None:
            output_dir = os.path.dirname(docx_file_path)
       
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
 
        # Determine the platform (Linux or Windows)
        system_platform = platform.system()
 
        # Build the command for LibreOffice (soffice)
        if system_platform == "Linux":
            # soffice_cmd = '/home/libreoffice/lib_office/opt/libreoffice24.2/program/soffice'  # Typically the command in Linux
            # soffice_cmd = '/home/libreoffice/lib/program/soffice'
            soffice_cmd = '/home/libreoffice/lib_office/opt/libreoffice24.2/program/soffice'
            # soffice_cmd = '/usr/bin/soffice'
            # soffice_cmd = '/opt/libreoffice24.2/program/soffice'
        elif system_platform == "Windows":
            soffice_cmd = 'C:/Program Files/LibreOffice/program/soffice.exe'
        else:
            raise OSError(f"Unsupported platform: {system_platform}")
 
        # Output PDF path (same as input file but with .pdf extension)
        output_pdf_path = os.path.join(output_dir, os.path.splitext(os.path.basename(docx_file_path))[0] + ".pdf")
        abs_docx_file_path = os.path.abspath(docx_file_path)
        abs_output_dir = os.path.dirname(abs_docx_file_path)
        # Construct the command to convert the file
        command = [
            soffice_cmd,
            "--headless",  # Run in headless mode (without GUI)
            "--convert-to", "pdf",  # Convert to PDF format
            "--outdir", f"{abs_output_dir}",  # Specify the output directory
            f"{abs_docx_file_path}"  # Input file path
        ]
 
       
        # Execute the command
        subprocess.run(command, check=True)
        print(f"Conversion successful: {output_pdf_path}")
        # logging.debug(f"Conversion successful: {output_pdf_path}")
        return output_pdf_path
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"LibreOffice failed to convert {docx_file_path} to PDF: {str(e)}")

In [17]:
# done
def create_search_index(index_name, search_service_endpoint, search_api_key):
    # search_service_name = "poc-openai-cogsrch"
    search_service_name = os.getenv('AZURE_SEARCH_SERVICE_NAME')
    # admin_key = AzureKeyCredential("TBpoXA6at3L38HsBWSRHnZWfstC07Txt7kDulY7ccDAzSeAHslm9")
    admin_key = AzureKeyCredential(f"{os.getenv('AZURE_SEARCH_API_KEY')}")
    # index_name = "srathore_bulk"
    # indexer_client = SearchIndexerClient(endpoint=f"https://{search_service_name}.search.windows.net", credential=admin_key)

    # Initialize SearchIndexClient
    search_service_endpoint = f"https://{search_service_name}.search.windows.net"
    index_client = SearchIndexClient(endpoint=search_service_endpoint, credential=admin_key)
    
    fields = [
        SimpleField(name="id", type="Edm.String", key=True),
        SearchableField(name="content", type="Edm.String", searchable=True, filterable=False, sortable=False),
        SimpleField(name="filepath", type="Edm.String"),
        SearchableField(name="title", type="Edm.String"),
        SimpleField(name="url", type="Edm.String"),
        SimpleField(name="page_number",type="Edm.Int32")
        #SimpleField(name="page_number", type="Edm.Int32")
        # VectorField(name="contentVector", dimensions=1536, vector_search_configuration="vector_config")
    ]

    index = SearchIndex(
        name=index_name,
        fields=fields,
        # vector_search_configurations=[
        #     {"name": "vector_config", "kind": "vector_search"}
        # ]
    )

    index_client.create_index(index)


In [18]:
# done
def pdf_image(file_path, img_path):
    seen_hashes = set()
    img_present = False
    
    file_ext = os.path.splitext(file_path)[1].lower()

    try:
        if file_ext == ".pdf":
            pdf_file = fitz.open(file_path)
            for page_index in range(len(pdf_file)):
                page = pdf_file[page_index]
                image_list = page.get_images(full=True)

                if image_list:
                    img_present = True

                for image_index, img in enumerate(image_list):
                    xref = img[0]
                    base_image = pdf_file.extract_image(xref)
                    image_bytes = base_image["image"]
                    image_ext = base_image["ext"]
                    img_hash = hashlib.md5(image_bytes).hexdigest()

                    if img_hash not in seen_hashes:
                        seen_hashes.add(img_hash)
                        image_name = f"pageno_{page_index+1}_{image_index+1}.{image_ext}"
                        image_path = os.path.join(img_path, image_name)
                        with open(image_path, "wb") as img_file:
                            img_file.write(image_bytes)

        elif file_ext == ".pptx":
            ppt = Presentation(file_path)
            print("PPT_Image_Extraction")
            for slide_index, slide in enumerate(ppt.slides):
                for shape_index, shape in enumerate(slide.shapes):
                    try :
                        if shape.shape_type == 13 and hasattr(shape, "image"):  # IMAGE shape type
                            img_present = True
                            img = shape.image
                            img_data = img.blob
                            img_hash = hashlib.md5(img_data).hexdigest()

                            if img_hash not in seen_hashes:
                                seen_hashes.add(img_hash)
                                ext = img.ext
                                image_name = f"pageno_{slide_index+1}_{shape_index+1}.{ext}"
                                image_path = os.path.join(img_path, image_name)
                                with open(image_path, "wb") as img_file:
                                    img_file.write(img_data)
                    except AttributeError as e:
                     print(f"Skipping shape on slide {slide_index+1}, shape {shape_index+1}: {str(e)}")                

        else:
            print(f"Unsupported file type: {file_ext}")
            return False

    except Exception as e:
        print(f"Error processing file: {e}")
        return False
    # remove_blank_images(img_path,threshold=5)
    # remove_small_images(img_path, min_size=5120)

    return img_present


In [19]:
# done
def connect_to_blob_storage(container_name, connection_string):

    connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    # connection_string = "DefaultEndpointsProtocol=https;AccountName=sapocopenai;AccountKey=H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw==;EndpointSuffix=core.windows.net"
    # container_name = "bulk-upload-qna"
    container_name = os.getenv('AZURE_BULK_UPLOAD_CONTAINER_NAME')

    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    return container_client


In [20]:
# done
async def load_and_split_file(path,img_there,temp_image_path):
    if path.endswith('.pdf'):
        loader = PyPDFLoader(path)
    elif path.endswith(('.doc', '.docx')):
        loader = UnstructuredWordDocumentLoader(path)
    elif path.endswith(('.ppt', '.pptx')):
        loader = UnstructuredPowerPointLoader(path)
    else:
        raise ValueError(f"Unsupported file format: {path}")
    
    # Load the document
    doc = loader.load()
    start_time_upload=time.time()
    # Split the document into chunks with the specified chunk size and overlap
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    split_docs = text_splitter.split_documents(doc)

    # Threshold for determining insufficient text content
    token_threshold = 20
    insufficient_text_pages = []
    has_text = False
    # img_docs

    # Check each chunk for text content length
    total_tokens = sum(len(chunk.page_content.split()) for chunk in split_docs)
    has_text = total_tokens > 20

    for idx, chunk in enumerate(split_docs):
        # Handle page/slide metadata based on file type
        # if len(chunk.page_content.split()) >= token_threshold:
        #     has_text = True
        if path.endswith(('.ppt', '.pptx')):
            # print("Break1")
            chunk.metadata["page"] = idx + 1
            # print("Break2")  # Using chunk index to assign slide number (starting from 1)
        else:
            chunk.metadata["page"] = chunk.metadata.get("page", 0) + 1

               # print("Img_docs",img_doc,"Img_Docs")
        # else:
        #     # print("Break3")
        #     # insufficient_text_pages.append(chunk.metadata["page"])   
        #     print("Break4")
    end_time_upload = time.time()
 
    execution_time_upload = end_time_upload - start_time_upload
    print(f"Time taken for chunking: {execution_time_upload:.6f} seconds")        
                 
    # for chunk in split_docs:
    #     chunk.metadata["page"] = chunk.metadata.get("page", 0) + 1
    #     if len(chunk.page_content.split()) >= token_threshold:
    #         has_text = True  # Mark that the document has text

    #            # print("Img_docs",img_doc,"Img_Docs")
    #     else:
            # insufficient_text_pages.append(chunk.metadata["page"])
            

    # scanned_doc_handler = scannned_document()
    # scanned_doc_handler = scannned_document()
    print("Break2")
    # Trigger OCR only if no meaningful text is found in the document

    if not has_text:
        # print(f"Document appears to be scanned with insufficient text. Pages: {insufficient_text_pages}")
        start_time_upload=time.time()
        output_folder = os.path.join("./temp_images", os.path.basename(path).split('.')[0])
        os.makedirs(output_folder, exist_ok=True)

        # Convert all pages of the document into images
        # await scanned_doc_handler.split_pdf_into_images(path, output_folder)

        combined_ocr_results = []
        # Process the images with OCR
        for image_file in sorted(os.listdir(output_folder)):
            image_path = os.path.join(output_folder, image_file)
            if os.path.isfile(image_path) and image_file.endswith(('.png', '.jpeg', '.jpg')):
                ocr_text = '' #scanned_doc_handler.extract_text_from_image(image_path)
                # ocr_text = scanned_doc_handler.extract_text_from_image(image_path)
                if ocr_text.strip():  # Skip empty OCR results
                    page_number = int(image_file.split('_')[1].split('.')[0])  # Extract page number from file name
                    combined_ocr_results.append({"content": ocr_text, "page": page_number})

        # Clean up temporary files
        shutil.rmtree(output_folder, ignore_errors=True)

        # Check if OCR results are empty
        if not combined_ocr_results:
            raise ValueError("OCR failed to extract text from the document. No data to index.")
        
        end_time_upload = time.time()
 
        execution_time_upload = end_time_upload - start_time_upload
        print(f"Time taken for scanned document chunking: {execution_time_upload:.6f} seconds")

        return combined_ocr_results
    else:
        print(f"Document contains sufficient text. Skipping OCR.")



    # Combine chunks by page for normal text documents
    img_docs = []
    if has_text:
        if img_there:
            start_time_upload=time.time()

            files = os.listdir(temp_image_path)  
            for j,i in enumerate(files):
                # print(j,i)
                # print("##&&&&&&&&&&&&&&&&&&")
                match = re.search(pattern, i)
                if match:
                    pageno = match.group(1)
                img_info, img_token_info = img_summary(os.path.join(temp_image_path, i))
                ###testing###### 
                # print(img_info)
                img_doc = Document(img_info, metadata={'source': os.path.basename(path), 'page': pageno})
                # print(img_doc)
                # img_docs = text_splitter.split_documents([img_doc])
                img_docs.extend(text_splitter.split_documents([img_doc]))
                os.remove(os.path.join(temp_image_path, i))
               
                # print("Img_docs",img_doc,"Img_Docs")
            end_time_upload = time.time()
 
            execution_time_upload = end_time_upload - start_time_upload
            print(f"Time taken for Images within document processing: {execution_time_upload:.6f} seconds")    
    combined_docs = []
    split_docs_by_page = defaultdict(list)
    img_docs_by_page = defaultdict(list)
    if img_there:
        for img_doc in img_docs:
            # page = img_doc.metadata["page"]
            page = int(img_doc.metadata["page"])
            img_docs_by_page[page].append(img_doc)
            # print(f"@@@@@@ Image Document on Page {page}: {img_doc} !!!!!!!")
        # Group chunks by page number
    
    for doc in split_docs:
        # page = doc.metadata["page"]
        page = int(doc.metadata["page"])
        split_docs_by_page[page].append(doc)
        # print(f"////// Split Document on Page {page}: {doc} ///////")

    # for page_number in sorted(map(int, split_docs_by_page.keys())):
    #     combined_docs.extend(split_docs_by_page[page_number])
    all_pages = sorted(set(map(int, split_docs_by_page.keys())).union(map(int, img_docs_by_page.keys())))
    # print(f"All Pages to Process: {all_pages}")

    for page_number in all_pages:
        if page_number in img_docs_by_page:
            combined_docs.extend(img_docs_by_page[page_number])
            # print(f"Adding Image Docs from Page {page_number}: {img_docs_by_page[page_number]}")
        if page_number in split_docs_by_page:
            combined_docs.extend(split_docs_by_page[page_number])
            # print(f"Adding Split Docs from Page {page_number}: {split_docs_by_page[page_number]}")

    return [{"content": chunk.page_content, "page": chunk.metadata["page"]} for chunk in combined_docs]


In [21]:
# done
async def create_index(user_name):
    container_name_bulk = os.getenv('AZURE_BULK_UPLOAD_CONTAINER_NAME')
    container_name = f"{container_name_bulk}" + user_name
    connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
    # connection_string = "DefaultEndpointsProtocol=https;AccountName=sapocopenai;AccountKey=H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw==;EndpointSuffix=core.windows.net"
    search_service_name = os.getenv("AZURE_SEARCH_SERVICE_NAME")
    search_service_endpoint = f"https://{search_service_name}.search.windows.net"
    # search_api_key ="TBpoXA6at3L38HsBWSRHnZWfstC07Txt7kDulY7ccDAzSeAHslm9"
    search_api_key = os.getenv('AZURE_SEARCH_API_KEY')
    print("231", search_api_key, "search_api_key")
    openai.api_key = os.getenv("OPENAI_API_KEY")
    # index_name = f"{user_name}_index"
    # index_name = f"{user_name.split('@')[0]}-index"
    # index_name = "mparag-index"
    # index_name = f"{user_name.split('@')[0]}-index"
    index_name1 = re.sub(r'[^a-zA-Z0-9]', '', user_name.split('@')[0])
    index_name = f"{index_name1}"
    print(index_name,"Index_Name")

    index_client = SearchIndexClient(endpoint=search_service_endpoint, credential=AzureKeyCredential(search_api_key))

    # Check if the index already exists
    index_exists = False
    try:
        index_exists = index_client.get_index(index_name) is not None
    except Exception:
        # Index does not exist, will create a new one
        pass

    # Create the index if it does not exist
    if not index_exists:
        create_search_index(index_name, search_service_endpoint, search_api_key)
                            
    search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name=index_name,
        credential=AzureKeyCredential(search_api_key)
    )
    container_client = connect_to_blob_storage(container_name, connection_string)
    # print("Hello1")
    blobs = container_client.list_blobs()
    # # print("Hello2")
    # STORAGE_ACCOUNT_NAME="sapocopenai"
    # STORAGE_ACCOUNT_KEY="H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw=="

    # temp_folder_path = os.path.join("..", "./temp")

    sharepoint_tmp_folder_path = os.path.join("sharepoint_tmp_folder", 'ssirish@dsi.com')
    tempfile_path = os.path.join("tempfile", 'ssirish@dsi.com')

    # Dynamically select the folder based on existence
    selected_folder_path = None

    if os.path.exists(sharepoint_tmp_folder_path):
        selected_folder_path = sharepoint_tmp_folder_path
        print(f"Using 'sharepoint_tmp_folder': {selected_folder_path}")
    elif os.path.exists(tempfile_path):
        selected_folder_path = tempfile_path
        print(f"Using 'tempfile': {selected_folder_path}")
        temp_folder_path = os.path.join("sharepoint_tmp_folder", 'ssirish@dsi.com')
    else:
        print("No temporary folders found.")
    # Handle the case where neither folder exists
        selected_folder_path = None  # or raise an exception, return a specific response, etc.

# Proceed with file processing if a folder was found
    if selected_folder_path:
        # Add your file processing logic here
        print(f"Processing files in: {selected_folder_path}")

    # temp_folder_path = "sharepoint_tmp_folder/adeorukhkar@dsi.com"
    # temp_image_path1="./temp_image"
    temp_image_path = os.path.join("./temp_image", 'ssirish@dsi.com')
    os.makedirs(temp_image_path, exist_ok=True)
    for file_name in os.listdir(selected_folder_path):
        print("inside for loop indexcreation")
        file_path = os.path.join(selected_folder_path, file_name)
        print(file_name,"Filename")
        
        # Only process valid files
        if os.path.isfile(file_path):
            container_name = os.getenv('AZURE_BULK_UPLOAD_CONTAINER_NAME')
            # container_name = "bulk-upload-qna"
            user_name = user_name
            title = os.path.splitext(file_name)[0]
            url = f"https://sapocopenai.blob.core.windows.net/{container_name}/{user_name}/{title}.pdf"
            #url2 = await generate_previewable_url(container_name,title)
            batch = []

            # Load and split the file into chunks
            # temp_image_path="./temp_image"
            temp_image_path = os.path.join("./temp_image", 'ssirish@dsi.com')
            os.makedirs(temp_image_path, exist_ok=True)
            img_there=pdf_image(file_path,temp_image_path)
            chunks = await load_and_split_file(file_path,img_there,temp_image_path)
            if os.path.exists(os.path.join("temp_image", 'ssirish@dsi.com')):
            # print("yes database folder")
                await asyncio.get_event_loop().run_in_executor(None, shutil.rmtree, os.path.join("temp_image", 'ssirish@dsi.com')) 
            
            for chunk in chunks:
                        page_no = chunk["page"]
                        document = {
                            "id": str(uuid.uuid4()),
                            "content":chunk["content"],
                            "title": title,
                            "url": url,
                            # "page_number": chunk["metadata"].get("page_number", 1)
                            #"page_number": chunk["page_no"],
                            "page_number": {page_no},
                            "page_number": chunk.get("page"),
                            "url": f"{url}#page={page_no}"
                            #"page_number": chunk.metadata.get("page_number", 1)  # Include page number
                            #"page_number": chunk.get("metadata", {}).get("page_number", 1)
                        }
                        batch.append(document)
                        # if len(batch) >= BATCH_SIZE:
                        #    search_client.upload_documents(batch)
                            # batch = []

        # Upload any remaining documents in the last batch
        search_client.upload_documents(batch)
            # if batch:
                # upload_chunks_in_batches(search_client, batch)

        # print(f"Processed and indexed file: {blob_name}")

    print(f"Index creation and data upload completed for user {user_name}")


In [22]:
# done
async def upload_file_to_azure(container_name, file_path, connection_string, directory_path):
    # Create a BlobServiceClient using the connection string
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    # logging.debug(blob_service_client, "blob_service_client")
    # Create a ContainerClient
    container_client = blob_service_client.get_container_client(container_name)
    # logging.debug(container_client, "container_client")
   
    # Create the container if it does not exist
    try:
        container_client.create_container()
    except Exception as e:
        print(f"Container {container_name} already exists.")
 
    # Get the filename from the file path
    blob_name = os.path.basename(file_path)

    if directory_path:
        blob_name = os.path.join(directory_path, blob_name).replace("\\", "/")

    # Create a BlobClient
    blob_client = container_client.get_blob_client(blob_name)
   
    # Upload the file
    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
        print(f"File {file_path} uploaded to container {container_name} as blob {blob_name}")
        print(f"{blob_client} blob_client after upload blob")
        # logging.debug(f"File {file_path} uploaded to container {container_name} as blob {blob_name}")
        # logging.debug(f"{blob_client} blob_client after upload blob")
 
async def generate_previewable_url(container_name, blob_name, expiry_hours=10):
    
    try:
        blob_service_client_prev = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
        # Create a BlobServiceClient
        # blob_service_client = BlobServiceClient(account_url=f"https://{blob_service_client.account_name}.blob.core.windows.net", credential=blob_service_client.account_key)
        print(blob_service_client_prev, "blob_service_client_prev")
        # logging.debug((blob_service_client_prev, "blob_service_client_prev"))
        # Set the expiration time for the SAS token
        sas_expiry_time = datetime.now().astimezone() + timedelta(hours=expiry_hours)
        print(sas_expiry_time, "sas_expiry_time")
        # logging.debug(sas_expiry_time, "sas_expiry_time")
        
        # Generate a SAS token for the blob
        sas_token = generate_blob_sas(
            account_name=blob_service_client_prev.account_name,
            container_name=container_name,
            blob_name=blob_name,
            account_key=blob_service_client_prev.credential.account_key,
            permission=BlobSasPermissions(read=True),  # Set read permissions
            expiry=sas_expiry_time,
            content_disposition="inline",# Force the file to open in the browser
            content_type="application/pdf"
        )
        # logging.debug(sas_token, "sas_token")
        print(sas_token, "sas_token")
        # content_settings = ContentSettings(content_type="application/pdf", content_disposition="inline") 
        # Update blob headers if not already set correctly (optional, for existing files)
        # blob_client_prev = blob_service_client_prev.get_blob_client(container=container_name, blob=blob_name)
        # logging.debug(blob_client_prev, "blob_client_prev 183")
        # blob_client_prev.set_http_headers(content_settings=content_settings)
        # logging.debug(blob_client_prev, "blob_client_prev 185")
        # Generate the full URL with the SAS token
        print(container_name, "container_name before blob_url", blob_name, "blob_name before blob_url", sas_token, "sas_token before blob_url")
        # logging.debug(container_name, "container_name before blob_url", blob_name, "blob_name before blob_url", sas_token, "sas_token before blob_url")
        blob_url = f"https://{blob_service_client_prev.account_name}.blob.core.windows.net/{container_name}/{blob_name}?{sas_token}"
        # blob_url = f"https://{blob_service_client_prev.account_name}.blob.core.windows.net/{container_name}/{blob_name}"
        print(blob_url)
        # logging.debug(blob_url, "blob_url 200" )
        return blob_url
    
    except Exception as e:
        print(f"Failed to generate the URL: {e}")
 

In [23]:
# done
def token_counter(file):
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY_East")
    AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE_East")
    openai_api_type = "azure"
    openai_api_version = "2023-12-01-preview" 
    openai_api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com" 
    openai_api_key = AZURE_OPENAI_API_KEY
    embeddings = OpenAIEmbeddings(openai_api_base = openai_api_base,
                              openai_api_type = "azure",
                              openai_api_key=openai_api_key, 
                              openai_api_version = openai_api_version, 
                              deployment="text-embedding-ada-002", 
                              chunk_size = 1)
    if file.endswith('.pdf'):
        loader = PyPDFLoader(file)
        doc = loader.load()
    if file.endswith('.txt'):
        loader = TextLoader(file)
        doc = loader.load()
    if file.endswith('.docx'):
        loader = Docx2txtLoader(file)
        doc = loader.load()
    if file.endswith('.pptx'):
        loader = UnstructuredPowerPointLoader(file)
        doc = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap  = 200)
    docs = text_splitter.split_documents(doc)
    input_str = ""
    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    for i in range(len(docs)):
        input_str = input_str + docs[i].page_content
    
    return len(enc.encode(input_str)) 


In [24]:
# import queue
# from spire.doc import *
# from spire.doc.common import *

# def docx_image(file_path, img_path):
#     try:
#         document = Document()
#         document.LoadFromFile(file_path)
#         images = []

#         nodes = queue.Queue()
#         nodes.put(document)

#         while not nodes.empty():
#             node = nodes.get()
#             for i in range(node.ChildObjects.Count):
#                 obj = node.ChildObjects[i]
#                 # Find the images
#                 if isinstance(obj, DocPicture):
#                     picture = obj
#                     # Append the image data to the list
#                     data_bytes = picture.ImageBytes
#                     images.append(data_bytes)
#                 elif isinstance(obj, ICompositeObject):
#                     nodes.put(obj)
#         for i, image_data in enumerate(images):
#             file_name = f"Image-{i}.png"
#             with open(os.path.join(img_path, file_name), 'wb') as image_file:
#                 image_file.write(image_data)

#         document.Close()
#         if len(images) > 0:
#             image_flag = True
#         return image_flag
#     except:
#         pass


In [25]:
# done
async def create_vectorstore(file, user_name, file_url):
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY_East")
    AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE_East")
    openai_api_type = "azure"
   # openai_api_version = "2023-03-15-preview"
    openai_api_version = "2023-12-01-preview"  
    openai_api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
    openai_api_key = AZURE_OPENAI_API_KEY
    embeddings = OpenAIEmbeddings(openai_api_base = openai_api_base,
                              openai_api_type = "azure",
                              openai_api_key=openai_api_key,
                              openai_api_version = openai_api_version,
                              deployment="text-embedding-ada-002",
                              #chunk_size = 1,
                            #   max_retries = 0)
                                )
    doc_flag = False
 
    if file.endswith('.pdf'):
        loader = PyPDFLoader(file)
        doc = loader.load()
 
    if file.endswith('.txt'):
        pdf_path = convert_docx_to_pdf(file, None)
        # logging.debug("pdf_path", pdf_path)
        loader = PyPDFLoader(pdf_path)
        # print(loader, "loader")
        doc = loader.load()
        # print(doc, "doc")
 
    if file.endswith('.docx'):
        doc_flag = True
        
        pdf_path = convert_docx_to_pdf(file, None)
        
        loader = PyPDFLoader(pdf_path)
        # print(loader, "loader")
        doc = loader.load()
        # print(doc, "doc")
        pdf_file_path = os.path.splitext(file)[0] + ".pdf"
        if os.path.exists(file):
            os.remove(file)
        file = pdf_file_path
 
    if file.endswith('.pptx'):
        pdf_path = convert_docx_to_pdf(file, None)
        # logging.debug("pdf_path", pdf_path)
        loader = PyPDFLoader(pdf_path)
        # print(loader, "loader")
        doc = loader.load()
        # print(doc, "doc")
       
    if file.endswith(".pdf"):
        outer_dict = {}
        for j, i in enumerate(doc):
            page_no = i.metadata["page"] + 1
            inner_dict = {"content" : i.page_content, "source_url": file_url, "page_url" : f"{file_url}#page={page_no}"}
            outer_dict[page_no] = inner_dict
            print("outer_dict",outer_dict)
       
        os.makedirs(os.path.join("temp_dir_2", user_name), exist_ok=True)
        with open(f'./temp_dir_2/{user_name}/{os.path.basename(os.path.splitext(file)[0] + ".json")}', 'w') as json_file:
            json.dump(outer_dict, json_file)
 
   # print("Doc Created")
    global count
    count += 1
    index_flag = False
 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 200)
    docs = text_splitter.split_documents(doc)
    
    for j, i in enumerate(docs):
        try:
            if file.endswith(".pdf"):
                page_number = i.metadata["page"]
                # print(page_number, "page_number")
                page_number = page_number + 1
                pdf_url_with_page = f"{file_url}#page={page_number}"
                # print(pdf_url_with_page, "pdf_url_with_page")
                i.metadata["page"] = pdf_url_with_page
                i.metadata["source"] = file_url
 
            if file.endswith(".txt"):
                i.metadata["source"] = os.path.basename(file)
        except:
            pass
 
    vector_store_doc = FAISS.from_documents(documents=docs, embedding=embeddings)
 
    # print("Docs created", docs)
   
    timestamp = datetime.now()
    # enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    input_str = ""
    for i in range(len(docs)):
        input_str = input_str + docs[i].page_content
 
    Docs = []
    ## Checking if image present and extracting information
    img_flag = False
    tempdir = "./tempfile"
    img_desc = ""
    img_tokens = 0
    pattern = r"pageno_(\d+)"
    pageno = ""
    vector_dict = {}
 
    os.makedirs(os.path.join(tempdir, 'ssirish@dsi.com',"images"), exist_ok=True)
    img_path = os.path.join(tempdir, 'ssirish@dsi.com',"images")
   
    if file.endswith('.pdf'):
        img_flag = pdf_image(file, img_path)
   
    # elif file.endswith('.pptx'):
    #     prs = Presentation(file)
    #     extractor = PPTExtractor(img_path)
    #     img_flag = extractor.iter_picture_shapes(prs)
 
    # elif file.endswith('.docx'):
    #     img_flag = docx_image(file, img_path)
 
    print(img_flag)
    start_time = time.time()
    if img_flag:
        files = os.listdir(img_path)  
        for j,i in enumerate(files):
            # print(j,i)
            match = re.search(pattern, i)
            if match:
                pageno = match.group(1)
            img_info, img_token_info = img_summary(os.path.join(img_path, i))
            ###testing######
            # print(img_info)
            img_doc = Document(img_info, metadata={'source': os.path.basename(file), 'page': pageno})
            # print(img_doc)
            img_docs = text_splitter.split_documents([img_doc])
            vector_dict[f"vector_store_{j}"] = FAISS.from_documents(documents=img_docs, embedding=embeddings)
            # print(img_docs)
            if j == 0:
                vector_store = vector_dict[f"vector_store_{j}"]
            else:
                vector_store.merge_from(vector_dict[f"vector_store_{j}"])
 
            img_desc = img_desc + img_info
            img_tokens = img_tokens + img_token_info
            
    shutil.rmtree(img_path)
    end_time = time.time()
    print("************************************************************************************************")
    # print(end_time - start_time)
 
    input_str += img_desc
    print("*******************************************************")
    # print(input_str)
    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    embedding_tokens = len(enc.encode(input_str))
    
    print("image_text_extracted")
    
    print("img_document_created")
    
    path = os.path.join("vector_stored", user_name, "faiss_index_{}".format(count))
 
    if img_flag:
        vector_store.merge_from(vector_store_doc)
        vector_store.save_local(path)
    else:
        vector_store_doc.save_local(path)
 
    if doc_flag:
        os.remove(file)
 
def extract_slide_content_with_numbers(pptx_path):
    presentation = Presentation(pptx_path)
    slide_data = []
 
    for i, slide in enumerate(presentation.slides):
        slide_number = i + 1
        slide_content = ""
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                slide_content += shape.text + "\n"
            elif shape.shape_type == 19:  # Shape type 19 corresponds to a table
                table = shape.table
                for row in table.rows:
                    for cell in row.cells:
                        slide_content += cell.text + "\t"
                    slide_content += "\n"
        slide_data.append({
            "slide_number": slide_number,
            "content": slide_content.strip()
        })
 
    return slide_data


In [36]:
# done - upload()
# async def upload():
try:
    # global img_count
    start_time_upload_total=time.time()

    files = "C:/Users/ssirish/Downloads"
    # files = await request.files
    # print(files, "upload files")
    tempdir = "./tempfile"
    # os.makedirs('ssirish@dsi.com', exist_ok=True)
    os.makedirs(os.path.join(tempdir, 'ssirish@dsi.com'), exist_ok=True)

    unstructured_extensions = (".pdf", ".docx", ".pptx", ".txt")
    stored_img_flag = False
    total_size = 0
    # unstructured_files = []
    
    #tempath = os.path.join(tempdir, 'ssirish@dsi.com', file.filename)
    folder_files = os.listdir(os.path.join(tempdir, 'ssirish@dsi.com'))
    print(folder_files)
    # folder_files = os.listdir(os.path.join(tempdir, output_folder))
    file_name = None
    

    file_count = 0
    csv_excel_file_count = 0
    csv_excel_flag = False
    img_flag = False
    unstructured_flag = False

    for file in files:
        file_count += 1
        print(f"Processing file: {files[file].filename}")
        
        if files[file].filename.endswith(unstructured_extensions):
            unstructured_flag = True

    print("File_count",file_count)
    
        
    for file in files:
        print(f"Processing file2: {files[file].filename}")
        img_flag = False
        csv_excel_flag = False
        if img_flag:
            _, file_extension = os.path.splitext(files[file].filename)
        
        user_dir = os.path.join(tempdir, 'ssirish@dsi.com')
        # user_dir = os.path.join(tempdir, output_folder)

        # Ensure the directory exists
        os.makedirs(user_dir, exist_ok=True)

        # Save the file
        file_path = os.path.join(user_dir, files[file].filename)
        await files[file].save(file_path)

        # Calculate the total size of all files in the directory
        total_size = 0

        # Iterate over all files in the user directory
    for file_name in os.listdir(user_dir):
        file_path = os.path.join(user_dir, file_name)
        
        # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            # Add the file size to the total
            total_size += os.path.getsize(file_path)
            print(total_size)

    if files[file].filename.endswith(unstructured_extensions):
        file_path_list = []
        files = os.listdir(os.path.join(tempdir, 'ssirish@dsi.com'))
        for file_name in files:
            file_path = os.path.join(os.path.join(tempdir, 'ssirish@dsi.com'), file_name)               
            file_path_list.append(file_path) 
        
        for i in file_path_list:
            if i.endswith('.docx')  or i.endswith('.txt'):
                pdf_path = convert_docx_to_pdf(i, None)
                if os.path.exists(i):
                    os.remove(i)
                i = pdf_path  # Update the file path to the PDF path
                print(i, "i 688")# if i.endswith('.docx') or i.endswith('.pptx') or i.endswith('.txt'):
            

        # Call create_index function for this file
        await create_index(user_name='ssirish@dsi.com')
        for i in file_path_list:
            if i.endswith('.pptx') :
                pdf_path = convert_docx_to_pdf(i, None)
                if os.path.exists(i):
                    os.remove(i)
                i = pdf_path
                print(i, "i 688")# if i.endswith('.docx') or i.endswith('.pptx') or i.endswith('.txt'):
            

            # source_container_name = "bulk-upload-qna"
            source_container_name = os.getenv('AZURE_BULK_UPLOAD_CONTAINER_NAME')
            connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
            # connection_string = "DefaultEndpointsProtocol=https;AccountName=sapocopenai;AccountKey=H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw==;EndpointSuffix=core.windows.net"
            await upload_file_to_azure(source_container_name, i, connection_string, 'ssirish@dsi.com')
            print("here 594")
            blob_name = 'ssirish@dsi.com' + "/" + os.path.basename(i)
        
            blob_folder = "blob/" + 'ssirish@dsi.com'

        source_container_name = os.getenv('AZURE_BULK_UPLOAD_CONTAINER_NAME')
        connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        # source_container_name = "bulk-upload-qna"
        # connection_string = "DefaultEndpointsProtocol=https;AccountName=sapocopenai;AccountKey=H30mVmhmyKpFmLUkFSKpDO3CUe+jbcG8aGZ8TgCRNTQj5Ac5HB+649BzYypyo9eW0W9BRy9Z0oFr+ASt6hAVYw==;EndpointSuffix=core.windows.net"
        # os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        for file_name in os.listdir(user_dir):
            file_path = os.path.join(user_dir, file_name)

            # Check if it's a file (not a directory)
            if os.path.isfile(file_path):
            # Upload the current file to Azure Blob Storage
                await upload_file_to_azure(source_container_name, file_path, connection_string, 'ssirish@dsi.com')
        end_time_upload_total = time.time()

        execution_time_upload = end_time_upload_total - start_time_upload_total
        print(f"Time taken for document Upload: {execution_time_upload:.6f} seconds")

    tempath = os.path.join(tempdir, 'ssirish@dsi.com', files[file].filename)
        
    print(csv_excel_flag, "excel_csv_flag")
    if (not csv_excel_flag) and (not img_flag):
        file_path_list = []
        scanned_file_path_list = []
        token_limit_crossed = False
        max_token_limit = 1000000
        total_tokens = 0
        files = os.listdir(os.path.join(tempdir, 'ssirish@dsi.com'))  
        print(files, "652")        

        for file_name in files:
            file_tokens = 0
            print(file_name, "filename inside upload")
            file_path = os.path.join(os.path.join(tempdir, 'ssirish@dsi.com'), file_name)
            # file_path = os.path.join(file_name)
            print(file_path, "file_path 659")

            file_tokens = token_counter(file_path)
            total_tokens = total_tokens + file_tokens
            
            if file_tokens < 10:
                scanned_file_path_list.append(file_path)
                print(scanned_file_path_list, "scanned_file_path_list")
            else:
                file_path_list.append(file_path)
                print(file_path_list, "file_path_list")

            if total_tokens > max_token_limit:
                token_limit_crossed = True
                shutil.rmtree(os.path.join(tempdir, 'ssirish@dsi.com'))
        
        if not token_limit_crossed:
            
            for i in file_path_list:
                if i.endswith('.docx') or i.endswith('.pptx') or i.endswith('.txt'):
                    pdf_path = convert_docx_to_pdf(i, None)
                    if os.path.exists(i):
                        # print("Hello122")
                        os.remove(i)
                    i = pdf_path
                    # print(i, "i 688")

                source_container_name = os.getenv('AZURE_TARGET_CONTAINER_NAME_FILE_UPLOAD')
                connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
                await upload_file_to_azure(source_container_name, i, connection_string, 'ssirish@dsi.com')
                print("here 594")
                blob_name = 'ssirish@dsi.com' + "/" + os.path.basename(i)
                file_url = await generate_previewable_url(source_container_name, blob_name)
                # file_url = await generate_target_uri(source_container_name,files[file].filename, "", 'ssirish@dsi.com')
                print("*****************file_url*************************")
               
            
                vector_store = await create_vectorstore(file = i, user_name = 'ssirish@dsi.com', file_url = file_url)                                        
                    
                if os.path.exists(i):
                    os.remove(i)
                    # print("file_path deleted", i)
            for j, i in enumerate(scanned_file_path_list):
                folder_path = os.path.join(tempdir, 'ssirish@dsi.com', os.path.splitext(os.path.basename(i))[0])
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                # await scannned_document().create_vstore(i, folder_path, 'ssirish@dsi.com')
                if os.path.exists(i):
                    os.remove(i)

    end_time_upload_total = time.time()

    execution_time_upload = end_time_upload_total - start_time_upload_total
    print(f"Time taken for document Upload: {execution_time_upload:.6f} seconds")
    # return jsonify({"status": "success", "test_purpose": "test_purpose"}), 200
except Exception as e:
    tempdir = "./tempfile"
    # remove_directory(os.path.join(tempdir, 'ssirish@dsi.com'))
    print("error occured:", e)
    # return jsonify(status='try_after_some_time', error=str(e)), 600


[]
error occured: string indices must be integers, not 'str'


In [ ]:
# upload()

<coroutine object upload at 0x00000211BF9F4440>